## Desafío

En este desafío, el servidor recibe un mensaje elegido por el usuario, codificado en base64, concatena un mensaje secreto con ese mensaje, y devuelve el resultado cifrado con AES en modo ECB y codificado en base64.

Es decir, si <mensaje> es el texto elegido por el usuario, el servidor enviará:



```
encode_base64(AES-ECB(decode_base64(<mensaje>)||<mensaje_secreto>))
```

donde:

* encode_base64() y decode_base64() denotan la codificación y decodificación en base64, respectivamente.
* AES-ECB() denota la aplicación de AES en modo ECB
* <mensaje_secreto> es un mensaje generado por el servidor.
* || denota concatenación.

El desafío consiste en descifrar el mensaje secreto.

Para obtener el texto cifrado, se debe hacer un requerimiento POST a una URL de la forma:


```
https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/<email>/encrypt
```
        
dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con un campo message que contiene el mensaje elegido, codificado en base64.

La respuesta será un texto cifrado codificado en base64, como se describió anteriormente.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar el campo requerido.

## Análisis
La clave para resolver este desafío consiste en comprender que se pueden enviar distintos textos elegidos, y analizar lo respondido por el servidor.

Una posible secuencia de pasos para resolver el desafío puede ser la siguiente:

1. Determinar la longitud del mensaje secreto. La longitud del texto cifrado será igual a la longitud del texto claro elegido, más la longitud del mensaje secreto, más la longitud del relleno. Para ello podemos enviar primero un mensaje vacío, luego uno de longitud 1, luego otro de longitud 2, etc. Cuando cambie la longitud del texto cifrado será porque la longitud de nuestro mensaje, sumada a la longitud del mensaje secreto, completó un múltiplo del tamaño de bloque, y habremos averiguado la longitud deseada.
2. Enviar un mensaje de 15 bytes. Esto significa que el primer bloque estará conformado por los 15 bytes que conocemos, y el primer byte del mensaje secreto, que no conocemos. Extraer el primer bloque del texto cifrado.
3. Enviar sucesivos mensajes de 16 bytes. Los primeros 15 bytes deben ser iguales a los 15 bytes del paso anterior. Variar el último byte hasta que el primer bloque del texto cifrado sea igual al obtenido en el paso anterior. El primer byte del mensaje secreto será, pues, el byte utilizado como último byte del texto claro correspondiente a ese bloque.
4. Repetir el paso 2, pero ahora enviando un bloque en el que el byte 15 es el byte que hemos encontrado.
5. Repetir el paso 3 para averiguar el segundo byte del mensaje secreto.
6. Descifrar el resto de los bytes del mensaje secreto de manera similar.




In [1]:
import requests
import base64
import string

In [2]:

def get_message(data):
  email = 'solujan@gmail.com'
  url = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/{email}/encrypt'
  payload = {'message':base64.b64encode(data.encode('ascii'))}
  # Solicitud
  request = requests.post(url, files=payload)
  # Body de la respuesta
  return b64_decode(request.content)

In [3]:
def print_hex(lista, bytes_):
  hex_list = ''
  for each in range(0,len(lista), 2):
    if each % bytes_ == 0:
      print(hex_list)
      hex_list = ''
    hex_list = hex_list + ' ' + lista[each:each+2]
  print(hex_list)



```
encode_base64(AES-ECB(decode_base64(<mensaje>)||<mensaje_secreto>))
```
donde:

* encode_base64() y decode_base64() denotan la codificación y decodificación en base64, respectivamente.
* AES-ECB() denota la aplicación de AES en modo ECB
* <mensaje_secreto> es un mensaje generado por el servidor.
* || denota concatenación.


In [4]:
def b64_decode(message_encrypted):
  message_decoded_from_base64 = base64.b64decode(message_encrypted)
  return message_decoded_from_base64.hex()

`Determinar la longitud del mensaje secreto. La longitud del texto cifrado será igual a la longitud del texto claro elegido, más la longitud del mensaje secreto, más la longitud del relleno. Para ello podemos enviar primero un mensaje vacío, luego uno de longitud 1, luego otro de longitud 2, etc. Cuando cambie la longitud del texto cifrado será porque la longitud de nuestro mensaje, sumada a la longitud del mensaje secreto, completó un múltiplo del tamaño de bloque, y habremos averiguado la longitud deseada.`

In [6]:
empty_str = get_message("")
starting_len = len(empty_str)
empty_str

'e7127e6d8c4ffc4a38600bb817f63c95a17bf583486e856e6bf8028324add5f0d48b5165a16fb1c17f660fc7560c353e7aa5197fad6b1d1137ffe470dc0b30f92693b62c6a53247006b52e0f364243eef4b08ec8e0f87e2d3144cd2062384a9f4cb30b6053cd2a782e713bd212befad8a97be41484fbc3232827ab749639f2755316d7701078dffb0035b5d52cf774484df72b37c44aeb3663afced5e2ee743033026aeaa2fcaaa5ef32614178e5a2219678104f9cdd54cf687df16c5a93eeb4ecb698d019b17b121951d9208f2e18e4a61c8a3611f7445941942930e4599660063bdb63e3aa3189d3b7a6592943a95eaf0f088b29ef811ea78e3fef3c55a1d6e50955b229f0995f21c7630fa26b52830652f4d06a1e543061a75bec047d2f7535cb18ed0eaa386862868e78ca8f18436c2a4c90a62d21ce89bcfbf488b3958b358d6a022742ce3514b35dcdbb5cb4708bc587937f5272a67c4938407e63d90ffee774e22f5d8a0b0705eea6ca7bbd07389508ca2c7b87a3de061853075caa3914220b2486502771ecb3c2513f92b90488878350594ad573f0245de7d7eeae59cc478eab65c06300426a2c4fd0bfef3df908d03916f03688f30360fd37bb032e51dbf47b80df918f9e9b1fb0ad97eb5e'

In [7]:
current_len = 0
data = ""
while starting_len >= current_len :
  current_str = get_message(data)
  current_len = len(current_str)
  print(f"\ndata: {data}\ncurrent str {current_str}\n len: {current_len}")
  data += "a"


data: 
current str e7127e6d8c4ffc4a38600bb817f63c95a17bf583486e856e6bf8028324add5f0d48b5165a16fb1c17f660fc7560c353e7aa5197fad6b1d1137ffe470dc0b30f92693b62c6a53247006b52e0f364243eef4b08ec8e0f87e2d3144cd2062384a9f4cb30b6053cd2a782e713bd212befad8a97be41484fbc3232827ab749639f2755316d7701078dffb0035b5d52cf774484df72b37c44aeb3663afced5e2ee743033026aeaa2fcaaa5ef32614178e5a2219678104f9cdd54cf687df16c5a93eeb4ecb698d019b17b121951d9208f2e18e4a61c8a3611f7445941942930e4599660063bdb63e3aa3189d3b7a6592943a95eaf0f088b29ef811ea78e3fef3c55a1d6e50955b229f0995f21c7630fa26b52830652f4d06a1e543061a75bec047d2f7535cb18ed0eaa386862868e78ca8f18436c2a4c90a62d21ce89bcfbf488b3958b358d6a022742ce3514b35dcdbb5cb4708bc587937f5272a67c4938407e63d90ffee774e22f5d8a0b0705eea6ca7bbd07389508ca2c7b87a3de061853075caa3914220b2486502771ecb3c2513f92b90488878350594ad573f0245de7d7eeae59cc478eab65c06300426a2c4fd0bfef3df908d03916f03688f30360fd37bb032e51dbf47b80df918f9e9b1fb0ad97eb5e
 len: 928

data: a
current str 7466add31ce6adcc05b5

In [22]:
message = get_message('aaaaaaaaaaaaaaa')
print_hex(message, 32)


 7e d4 71 7e af 46 82 54 53 08 ad fa ec c0 0f d7
 ca 76 f5 10 f9 06 35 14 f2 d3 3d 1e e4 77 d2 a9
 43 27 ec 68 3a 1c b5 f2 82 fd 26 dc ff 3e cf e0
 ed 7e d8 4f d4 62 88 99 77 f5 76 30 b7 a1 de b6
 02 4a 85 07 1d 9a 91 1a a8 20 82 69 01 e6 18 34
 6c d5 60 d1 9a 47 a0 78 b9 da f7 81 50 24 b4 b4
 f9 23 7c bc c7 57 ee 50 13 15 bf f7 f0 61 6a 73
 cb a2 4b e3 fb 46 2f 75 5e 22 4b 74 02 c4 1f 19
 00 ee 2e 7d 91 7d 69 98 bc ea 5a c2 b5 92 44 ed
 38 b5 c4 8f 53 e6 f1 14 1f 7c 27 44 5a a1 a2 f5
 71 d7 7c ce e1 b6 f1 e8 80 e0 32 5d 84 21 3d 75
 38 70 d4 e3 3e e7 6a 90 b7 5d 74 de ae 49 e0 36
 c2 1c 40 ca da 4f cb 62 43 d5 13 8c fa 62 ce 95
 7f 96 50 89 72 90 ba ea 8e bd e3 a5 37 72 d8 e3
 86 a5 fd fe 25 17 11 3d d9 3e f4 99 4e d3 ca 78
 ab 8f fe 24 19 51 63 a9 50 a2 2c ea 17 79 22 9a
 b5 b1 ae d7 b9 2f 9d 11 43 95 45 60 b2 c3 dc 90
 d9 50 e9 ea 8e 4a a8 2a f3 3b 4c a4 da da 2d 88
 a0 34 62 fd 4a 98 66 10 08 b0 3e 14 88 f7 e9 64
 19 93 73 8b 31 57 7a a9 48 93 3b 2e 93 b8 fa 8d
 41 8e d8 42 54 03 

Determinar la longitud del mensaje secreto. La longitud del texto cifrado será igual a la longitud del texto claro elegido, más la longitud del mensaje secreto, más la longitud del relleno. Para ello podemos enviar primero un mensaje vacío, luego uno de longitud 1, luego otro de longitud 2, etc. Cuando cambie la longitud del texto cifrado será porque la longitud de nuestro mensaje, sumada a la longitud del mensaje secreto, completó un múltiplo del tamaño de bloque, y habremos averiguado la longitud deseada.

In [74]:
# el mensaje se concatena con el mensaje secreto
#  AES-ECB('aaaaaaaaaaaaaaa' || mensaje_secreto)
# Y este mensaje completa el bloque
mensaje_aux = 'aaaaaaaaaaaaaaa'
mensaje_aux

'aaaaaaaaaaaaaaa'

In [75]:
# el mensaje tiene un largo de 20 bytes
len_mensaje_bytes = len(mensaje_aux)
len_mensaje_bytes

15

In [76]:
len_full_mensaje_bytes = len(message) // 2
len_full_mensaje_bytes

480

In [77]:
block_size_bytes = 16
len_mensaje_secreto_bytes = len_full_mensaje_bytes - len_mensaje_bytes - block_size_bytes
len_mensaje_secreto_bytes

449

In [81]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

Enviar sucesivos mensajes de 16 bytes. Los primeros 15 bytes deben ser iguales a los 15 bytes del paso anterior. Variar el último byte hasta que el primer bloque del texto cifrado sea igual al obtenido en el paso anterior. El primer byte del mensaje secreto será, pues, el byte utilizado como último byte del texto claro correspondiente a ese bloque.

In [115]:
bloque_inicial = 'aaaaaaaaaaaaaaa'
letter = ''
for index in range(len(bloque_inicial), -1, -1):
  bloque_aux = bloque_inicial[:index] + letter
  print(f'Bloque: {bloque_aux} - len {len(bloque_aux)} - letter: {letter}')
  letter_get = get_next_letter(bloque_aux, get_message(bloque_inicial[:index])[:32],32)
  letter = letter + letter_get


Bloque: aaaaaaaaaaaaaaa - len 15 - letter: 
La letra es: S
Bloque: aaaaaaaaaaaaaaS - len 15 - letter: S
La letra es: h
Bloque: aaaaaaaaaaaaaSh - len 15 - letter: Sh
La letra es: e
Bloque: aaaaaaaaaaaaShe - len 15 - letter: She
La letra es:  
Bloque: aaaaaaaaaaaShe  - len 15 - letter: She 
La letra es: s
Bloque: aaaaaaaaaaShe s - len 15 - letter: She s
La letra es: t
Bloque: aaaaaaaaaShe st - len 15 - letter: She st
La letra es: o
Bloque: aaaaaaaaShe sto - len 15 - letter: She sto
La letra es: o
Bloque: aaaaaaaShe stoo - len 15 - letter: She stoo
La letra es: d
Bloque: aaaaaaShe stood - len 15 - letter: She stood
La letra es:  
Bloque: aaaaaShe stood  - len 15 - letter: She stood 
La letra es: o
Bloque: aaaaShe stood o - len 15 - letter: She stood o
La letra es: n
Bloque: aaaShe stood on - len 15 - letter: She stood on
La letra es:  
Bloque: aaShe stood on  - len 15 - letter: She stood on 
La letra es: t
Bloque: aShe stood on t - len 15 - letter: She stood on t
La letra es: h
Bloque: Sh

In [117]:
def get_next_letter(mensaje:str, bloque:str, count:int):
  for letter in string.printable:
    new_message = mensaje + letter
    #print(f'Nuevo mensaje: {new_message}')
    new_message_enc = get_message(new_message)
    #print(f'Comparo: {bloque} - {new_message_enc[:32]}')
    if new_message_enc[:count] == bloque:
      print(f'La letra es: {letter}')
      return letter
  print(f'Failed!')

In [179]:
bloque_inicial = 'aaaaaaaaaaaaaaa'
text_decrypt = ''
for block_size in range(32, 500, 32):
  letter = ''
  for indice in range(len(bloque_inicial), -1, -1):
    bloque_aux = bloque_inicial[:indice] + text_decrypt + letter
    letter_get = get_next_letter(bloque_aux, get_message(bloque_inicial[:indice])[:block_size],block_size)
    letter = letter + letter_get
    print(f'Bloque: \'{bloque_aux}\' - len {len(bloque_aux)} - letter: {letter}')
  text_decrypt = text_decrypt + letter
  print(f'text_decrypt: \'{text_decrypt}\'')

La letra es: S
Bloque: 'aaaaaaaaaaaaaaa' - len 15 - letter: S
La letra es: h
Bloque: 'aaaaaaaaaaaaaaS' - len 15 - letter: Sh
La letra es: e
Bloque: 'aaaaaaaaaaaaaSh' - len 15 - letter: She
La letra es:  
Bloque: 'aaaaaaaaaaaaShe' - len 15 - letter: She 
La letra es: s
Bloque: 'aaaaaaaaaaaShe ' - len 15 - letter: She s
La letra es: t
Bloque: 'aaaaaaaaaaShe s' - len 15 - letter: She st
La letra es: o
Bloque: 'aaaaaaaaaShe st' - len 15 - letter: She sto
La letra es: o
Bloque: 'aaaaaaaaShe sto' - len 15 - letter: She stoo
La letra es: d
Bloque: 'aaaaaaaShe stoo' - len 15 - letter: She stood
La letra es:  
Bloque: 'aaaaaaShe stood' - len 15 - letter: She stood 
La letra es: o
Bloque: 'aaaaaShe stood ' - len 15 - letter: She stood o
La letra es: n
Bloque: 'aaaaShe stood o' - len 15 - letter: She stood on
La letra es:  
Bloque: 'aaaShe stood on' - len 15 - letter: She stood on 
La letra es: t
Bloque: 'aaShe stood on ' - len 15 - letter: She stood on t
La letra es: h
Bloque: 'aShe stood on t' 

In [192]:
song = '''She stood on the tracks\nWaving her arms\nLeading me to that third rail shock\nQuick as a wink\nShe changed her mind\n\nShe gave me a night\nThat's all it was\nWhat will it take until I stop\nKidding myself\nWasting my time\n\nThere's nothing else I can do\n'Cause I'm doing it all for Leyna\nI don't want anyone new\n'Cause I'm living it all for Leyna\nThere's nothing in it for you\n'Cause I'm giving it all to Leyna'''
print(len(song))
message = get_message(song)
print_hex(message, 32)

399

 e7 12 7e 6d 8c 4f fc 4a 38 60 0b b8 17 f6 3c 95
 a1 7b f5 83 48 6e 85 6e 6b f8 02 83 24 ad d5 f0
 d4 8b 51 65 a1 6f b1 c1 7f 66 0f c7 56 0c 35 3e
 7a a5 19 7f ad 6b 1d 11 37 ff e4 70 dc 0b 30 f9
 26 93 b6 2c 6a 53 24 70 06 b5 2e 0f 36 42 43 ee
 f4 b0 8e c8 e0 f8 7e 2d 31 44 cd 20 62 38 4a 9f
 4c b3 0b 60 53 cd 2a 78 2e 71 3b d2 12 be fa d8
 a9 7b e4 14 84 fb c3 23 28 27 ab 74 96 39 f2 75
 53 16 d7 70 10 78 df fb 00 35 b5 d5 2c f7 74 48
 4d f7 2b 37 c4 4a eb 36 63 af ce d5 e2 ee 74 30
 33 02 6a ea a2 fc aa a5 ef 32 61 41 78 e5 a2 21
 96 78 10 4f 9c dd 54 cf 68 7d f1 6c 5a 93 ee b4
 ec b6 98 d0 19 b1 7b 12 19 51 d9 20 8f 2e 18 e4
 a6 1c 8a 36 11 f7 44 59 41 94 29 30 e4 59 96 60
 06 3b db 63 e3 aa 31 89 d3 b7 a6 59 29 43 a9 5e
 af 0f 08 8b 29 ef 81 1e a7 8e 3f ef 3c 55 a1 d6
 e5 09 55 b2 29 f0 99 5f 21 c7 63 0f a2 6b 52 83
 06 52 f4 d0 6a 1e 54 30 61 a7 5b ec 04 7d 2f 75
 e7 42 4d 2b 08 60 ca 42 29 1b 76 b4 55 9c af 47
 41 8e d8 42 54 03 12 07 3e b4 9f 79 63 5f 21 00
 55 13 d5 d0 35

In [190]:
message = get_message('')
print_hex(message, 32)


 e7 12 7e 6d 8c 4f fc 4a 38 60 0b b8 17 f6 3c 95
 a1 7b f5 83 48 6e 85 6e 6b f8 02 83 24 ad d5 f0
 d4 8b 51 65 a1 6f b1 c1 7f 66 0f c7 56 0c 35 3e
 7a a5 19 7f ad 6b 1d 11 37 ff e4 70 dc 0b 30 f9
 26 93 b6 2c 6a 53 24 70 06 b5 2e 0f 36 42 43 ee
 f4 b0 8e c8 e0 f8 7e 2d 31 44 cd 20 62 38 4a 9f
 4c b3 0b 60 53 cd 2a 78 2e 71 3b d2 12 be fa d8
 a9 7b e4 14 84 fb c3 23 28 27 ab 74 96 39 f2 75
 53 16 d7 70 10 78 df fb 00 35 b5 d5 2c f7 74 48
 4d f7 2b 37 c4 4a eb 36 63 af ce d5 e2 ee 74 30
 33 02 6a ea a2 fc aa a5 ef 32 61 41 78 e5 a2 21
 96 78 10 4f 9c dd 54 cf 68 7d f1 6c 5a 93 ee b4
 ec b6 98 d0 19 b1 7b 12 19 51 d9 20 8f 2e 18 e4
 a6 1c 8a 36 11 f7 44 59 41 94 29 30 e4 59 96 60
 06 3b db 63 e3 aa 31 89 d3 b7 a6 59 29 43 a9 5e
 af 0f 08 8b 29 ef 81 1e a7 8e 3f ef 3c 55 a1 d6
 e5 09 55 b2 29 f0 99 5f 21 c7 63 0f a2 6b 52 83
 06 52 f4 d0 6a 1e 54 30 61 a7 5b ec 04 7d 2f 75
 35 cb 18 ed 0e aa 38 68 62 86 8e 78 ca 8f 18 43
 6c 2a 4c 90 a6 2d 21 ce 89 bc fb f4 88 b3 95 8b
 35 8d 6a 02 27 42 